# Managing the Quality Metric of Global Ecological Footprint

> Managing the Quality Metric of Global Ecological Footprint

- author: Victor Omondi
- toc: true
- comments: true
- categories: [classification, machine-learning]
- image: images/mqmgef-shield.png

# Overview

## Machine Learning: Classification - Managing the Quality Metric of Global Ecological Footprint


The dataset used  was obtained from the National Footprint and Biocapacity Accounts. It provides Ecological Footprint per capita data for years 1961-2016 in global hectares (gha). The National Footprint and Biocapacity Accounts (NFAs) measure the ecological resource use and resource capacity of nations from 1961 to 2016. The calculations in the National Footprint and Biocapacity Accounts are primarily based on United Nations data sets.

We will use the data to classify and predict the quality metrics (qascore) of the ecological footprint data for the different countries. This data includes total and per capita national biocapacity, the ecological footprint of consumption, the ecological footprint of production and total area in hectares.

Data Source: https://data.world/footprint/nfa-2019-edition

# Libraries

In [41]:
import warnings

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use("ggplot")

from sklearn.utils import shuffle
from sklearn.preprocessing import (LabelEncoder, 
                                   MinMaxScaler)
from sklearn.model_selection import (cross_val_score, 
                                     KFold, 
                                     LeaveOneOut, 
                                     StratifiedKFold, 
                                     train_test_split)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score,
                             confusion_matrix, 
                             f1_score, 
                             precision_score, 
                             recall_score)

from imblearn.over_sampling import SMOTE

In [5]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

# Linear Classification and Logistic Regression

We will Explore linear classification.

In machine learning, classification is a supervised method of segmenting data points into various labels or classes. Unlike regression, the target variable in a classification problem is discrete. Each data point used in training classification models must have a corresponding label in order for the characteristics and patterns in the classes to be learnt appropriately. Classification can either be binary - identifying that a given email is spam or not or, multi-class - classifying a fruit as orange, mango or banana.

## Introduction

Every year people demand more from nature than it can regenerate. Individuals, communities and government leaders use ecological footprint data to better manage limited resources, reduce economic risk, and improve well-being. The Dataset provides Ecological Footprint per capita data for years 1961-2016 in global hectares (gha). Ecological Footprint is a measure of how much area of biologically productive land and water an individual, population, or activity requires to produce all the resources it consumes and to absorb the waste it generates, using prevailing technology and resource management practices. The Ecological Footprint is measured in global hectares. Since trade is global, an individual or country's Footprint tracks area from all over the world. 

Apart from predicting numeric values, another important supervised machine learning method is classification and it involves predicting classes (either binary or multinomial classes). In this section, we will cover how to measure performances of class prediction, linear classification methods and non-linear/tree-based methods. We’ll also focus on strategies for applying a successful classification model like interpretability-accuracy trade-off, class and imbalance.

The National Footprint and Biocapacity Accounts (NFAs) measure the ecological resource use and resource capacity of nations from 1961 to 2016. The calculations in the National Footprint and Biocapacity Accounts are primarily based on United Nations data sets, including those published by the Food and Agriculture Organization, United Nations Commodity Trade Statistics Database, and the UN Statistics Division, as well as the International Energy Agency. In this project, we will use this data to classify and predict the quality metrics (qascore) of the ecological footprint data for the different countries. This data includes total and per capita national biocapacity, the ecological footprint of consumption, the ecological footprint of production and total area in hectares.

Data Source: https://data.world/footprint/nfa-2019-edition

## Linear Classification and Logistic Regression

In machine learning, classification is a supervised method of segmenting data points into various labels or classes. Unlike regression, the target variable in a classification problem is discrete. Each data point used in training classification models must have a corresponding label in order for the characteristics and patterns in the classes to be learnt appropriately. Classification can either be binary - identifying that a given email is spam or not or, multi-class - classifying a fruit as orange, mango or banana.

### Linear classifiers and the importance of class probabilities

For simplicity, we define a linear classifier as a binary classifier that separates two classes (positive and negative class) using a linear separator by computing a linear combination of the features and comparing against a set threshold.

### Logistic Regression: Sigmoid, logit and the log-likelihood

Logistic regression is a linear algorithm that can be used for binary or multiclass classification. It is a discriminative classifier that estimates the probability that an instance belongs to a class using an s-shape function curve called the sigmoid function. The predicted values obtained after using a linear equation on the predictors by applying logistic regression can fall in the range of negative infinity to positive infinity. The sigmoid maps these results by shrinking the value to fall between 0 and 1.  We can say that we use the sigmoid function to transform linear regression into logistic regression.

$$
sigmoid\ \sigma \ (x) = \frac{1}{1+e^{-x}} 
$$

![image.png](datasets/images/sigmoid-curve.png "sigmoid-curve.png")

The sigmoid function can be applied to a linear equation,

$$
z = \beta_0 + \beta_{1}x
$$

to obtain values h between 0 and 1 such that

$$
h = \sigma(z) = \frac{1}{1 + e^{-z}} = \frac{1}{1 + e^{-{\beta_0 + \beta_{1}x}}}
$$

For a binary classification task with classes A and B, if a threshold is set for 0.5 and the probability of an instance belonging to a class is $p$, we can say that if $p < 0.5$ the instance if of class A while it is of class B is $p > 0.5$. 

Also known as the log of odds, logit is the logarithm of odds ratio where the odds ratio is the probability that an event occurs divided by the probability that the event does not occur. Logit is the inverse of the sigmoid such that it maps values from negative infinity to positive infinity.

$$
\log{it}(p) = \log(\frac{p}{1 - p})
$$

> Note: Recall that in linear regression, we minimized the sum of squared errors SSE; in logistic regression, the log-likelihood is maximized.

In [6]:
df = pd.read_csv("datasets/raw/NFA 2019 public_data.csv")
df.head()

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72186 entries, 0 to 72185
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         72186 non-null  object 
 1   year            72186 non-null  int64  
 2   country_code    72186 non-null  int64  
 3   record          72186 non-null  object 
 4   crop_land       51714 non-null  float64
 5   grazing_land    51714 non-null  float64
 6   forest_land     51714 non-null  object 
 7   fishing_ground  51713 non-null  float64
 8   built_up_land   51713 non-null  float64
 9   carbon          51713 non-null  float64
 10  total           72177 non-null  float64
 11  QScore          72185 non-null  object 
dtypes: float64(6), int64(2), object(4)
memory usage: 6.6+ MB


In [8]:
df.isnull().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

The dataset has a lot of missing values from `crop_land:carbon` columns

### distribution of target variable

In [9]:
df.QScore.value_counts()

3A    51481
2A    10576
2B    10096
1B       16
1A       16
Name: QScore, dtype: int64

### Handling Missing Values

For simplicity, we will drop the rows with missing values.

In [10]:
df = df.dropna()
df.isnull().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [11]:
df.QScore.value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

An obvious change in our target variable after removing the missing values is that there are only three classes left. From the distribution of the 3 classes, we can see that there is an obvious imbalance between the classes. There are methods that can be applied to handle this imbalance such as oversampling and undersampling.

- Oversampling involves increasing the number of instances in the class with fewer instances
- Undersampling involves reducing the data points in the class with more instances.
For now, we will convert this to a binary classification problem by combining class '2A' and '1A'.

In [12]:
df['QScore'] = df.QScore.replace(['1A'], '2A')
df.QScore.value_counts()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


3A    51473
2A      240
Name: QScore, dtype: int64

In [13]:
df_2A = df[df.QScore=='2A']
df_3A = df[df.QScore=='3A'].sample(350)
data_df = df_2A.append(df_3A)
data_df.sample(10)

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
19172,Equatorial Guinea,2016,61,EFConsPerCap,2.113976e-01,2.680246e-02,0.165406213,8.998100e-02,2.247772e-02,1.359850e+00,1.875915e+00,2A
42455,Morocco,2016,143,EFProdTotGHA,8.723089e+06,5.623982e+06,2861022.894,4.989264e+06,9.241300e+05,2.057104e+07,4.369253e+07,2A
61614,Togo,1996,217,EFConsPerCap,3.370050e-01,7.458356e-02,0.511850313,8.418123e-02,1.916901e-02,1.200963e-01,1.146885e+00,3A
62670,Trinidad and Tobago,2016,220,EFConsPerCap,4.275409e-01,1.918018e-01,0.302139636,1.047238e-01,1.222536e-03,7.350750e+00,8.378179e+00,2A
22016,Djibouti,2016,72,AreaPerCap,2.122392e-03,1.804033e+00,0.005942697,2.446057e-01,2.647769e-02,0.000000e+00,2.083182e+00,2A
15481,Azerbaijan,2016,52,AreaTotHA,2.240000e+06,2.533000e+06,1165620,3.930000e+05,3.005230e+05,0.000000e+00,6.632143e+06,3A
63815,Turkey,2014,223,EFConsTotGHA,6.480075e+07,7.677842e+06,25286823.19,2.289002e+06,2.594082e+06,1.484026e+08,2.510511e+08,3A
62673,Trinidad and Tobago,2016,220,EFProdTotGHA,6.071428e+04,8.287973e+03,129192.8608,8.413697e+04,1.668713e+03,1.216205e+07,1.244605e+07,2A
57336,South Africa,2016,202,EFProdPerCap,2.073526e-01,1.934319e-01,0.26312955,8.346993e-02,2.523715e-02,2.609595e+00,3.382216e+00,2A
14319,Cuba,1979,49,EFProdTotGHA,3.786266e+06,9.759010e+05,1343688.028,1.019392e+06,3.434420e+05,1.131251e+07,1.878120e+07,3A


In [15]:
data_df = shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape

(590, 12)

In [16]:
data_df.QScore.value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

### More Data Preprocessing

In [17]:
data_df = data_df.drop(columns=['country_code', 'country', 'year'])
X = data_df.drop(columns='QScore')
y = data_df['QScore']

### split the data into training and testing sets

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0)
y_train.value_counts()

3A    248
2A    165
Name: QScore, dtype: int64

There is still an imbalance in the class distribution. For this, we use SMOTE only on the training data to handle this.


### encode categorical variable

In [21]:
encoder = LabelEncoder()
X_train['record'] = encoder.fit_transform(X_train.record)
X_test['record'] = encoder.fit_transform(X_test.record)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
smote = SMOTE(random_state=1)
X_train_balanced, y_balanced = smote.fit_sample(X_train, y_train)

In [26]:
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(X_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns=X_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = X_train_balanced.record
normalised_train_df.head()

,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,record
0,1.403662e-03,9.455794e-04,7.306580e-03,6.338967e-03,2.896524e-03,0.000000e+00,4.311340e-03,1
1,7.171136e-04,3.115198e-04,2.184420e-05,2.705514e-05,9.561281e-04,0.000000e+00,2.558996e-04,3
2,7.747294e-04,5.145269e-04,2.713808e-05,3.013558e-04,5.157063e-03,2.879969e-02,1.600361e-02,7
3,2.246356e-10,6.907430e-12,3.324145e-11,4.108555e-10,1.122788e-09,3.100424e-11,1.705827e-10,4
4,2.040465e-10,5.473459e-11,8.728030e-11,2.364081e-12,1.389536e-10,1.330914e-10,1.414858e-10,4


In [28]:
X_test = X_test.reset_index(drop=True)
normalised_test_df = scaler.fit_transform(X_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=X_test.drop(columns=['record']).columns)
normalised_test_df['record'] = X_test.record
normalised_test_df.head()

,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,record
0,2.219728e-10,1.365066e-11,2.513993e-10,4.729084e-11,1.595866e-10,1.712429e-10,1.692207e-10,4
1,6.794621e-11,2.093708e-10,1.893583e-10,3.381674e-10,6.695475e-11,0.000000e+00,5.148909e-11,0
2,9.701673e-04,5.205637e-03,4.199154e-03,4.157862e-03,1.223763e-03,0.000000e+00,1.094954e-03,1
3,7.426252e-04,1.505536e-03,2.635490e-03,3.494542e-04,5.976756e-04,4.058477e-05,5.527608e-04,7
4,9.465349e-04,6.742536e-04,1.152870e-03,1.089355e-03,9.933233e-04,5.395759e-05,4.245477e-04,5


### Logistic Regression

In [30]:
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

LogisticRegression()

# Measuring Classification Performance

We will explore cross validation techniques used by data scientist to avoid overfitting and enable generalization.

## Cross-validation and accuracy

Cross Validation (CV) is a well known and trusted method applied to avoid overfitting and enable generalization. Although there are different techniques used in performing cross validation, the fundamental concept involves partitioning the dataset into a number of subsets, holding out a set for evaluation then training the model on the other sets. This gives a more reliable estimate of how the model performs across different training sets because it provides an average score across different training samples used. The only drawback with cross validation is that it takes more time and computational resources however, the gain obtained in having a better model is very well worth this cost. **K-Fold cross validation**, **Stratified K-Fold cross validation** and **Leave One Out Cross Validation (LOOCV)** are some cross validation techniques.



In [33]:
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv=5, scoring='f1_macro')
scores

array([0.549955  , 0.47469388, 0.5437788 , 0.5555102 , 0.53301887])

### K-Fold Cross Validation

This technique is called K-Fold because the data is split into K equal groups.  If $k = 5$ a 5-fold cross validation can be performed such that the data is split into $k_1$, $k_2$, $k_3$, $k_4$ and $k_5$. The model is trained on $k_2 - k_5$ and evaluated on $k_1$ then repeated $k$ times until every group is used to train and test the model. 

![image.png](datasets/images/kfold.png "kfold.png")

In [36]:
kf = KFold(n_splits=5)
kf.split(normalised_train_df)
f1_scores = []

# run for every split
for train_index, test_index in kf.split(normalised_train_df):
    X_train_k, X_test_k = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
    y_train_k, y_test_k = y_balanced[train_index], y_balanced[test_index]
    model = LogisticRegression().fit(X_train_k, y_train_k)
    f1_scores.append(
        f1_score(y_true=y_test_k, y_pred=model.predict(X_test_k), pos_label='2A')*100
    )
f1_scores

[55.35714285714286,
 50.90909090909091,
 48.48484848484849,
 58.119658119658126,
 0.0]

### Stratified K-Fold Cross Validation

Stratified K-Fold cross validation ensures that in every fold, there is an equal proportion of each target class to obtain a good representation of the data and avoid imbalance and biased results. For example, if there are two target classes $t_1$ and $t_2$ with equal distribution in the data, it is best to ensure that the folds also have the same distribution.

In [38]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores_skf = []
for train_index, test_index in skf.split(normalised_train_df, y_balanced):
    X_train_skf, X_test_skf = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
    y_train_skf, y_test_skf = y_balanced[train_index], y_balanced[test_index]
    model = LogisticRegression().fit(X_train_skf, y_train_skf)
    f1_scores_skf.append(
        f1_score(y_true=y_test_skf, y_pred=model.predict(X_test_skf), pos_label='2A')
    )
f1_scores_skf

[0.5714285714285714,
 0.5742574257425743,
 0.5242718446601942,
 0.45161290322580644,
 0.5263157894736842]

### Leave One Out Cross Validation (LOOCV)

In this method, one instance is left out and used as the test set while the model is trained on $N-1$ data points where $N$ is the number of data points. This means that the number of instances and folds are equal.

In [40]:
loo = LeaveOneOut()
scores_loo = cross_val_score(
    LogisticRegression(), normalised_train_df, y_balanced, cv=loo, scoring='f1_macro'
)
average_score_loo = scores_loo.mean()
average_score_loo

0.532258064516129

## Confusion Matrix, Precision-Recall, ROC curve and the F1-score

Accuracy, precision, recall, F1-score and many others are evaluation metrics used in measuring the performance of classification models. We will discuss these metrics.

### Confusion Matrix

It is an $N$ x $N$ matrix that gives a summary of the correct and incorrect predicted classification results for the N target classes. The values in the diagonal of the matrix represent the number of correctly predicted classes while every other cell in the matrix indicates the misclassified classes. This means that the more predicted values that fall in the diagonal, the better the model. True positive, false positive, true negative and false negative are terms used when interpreting a confusion matrix.

![image.png](datasets/images/confusion-matrix.png "confusion-matrix.png")

#### True Positive (TP): 
This is a correct classification where the predicted value is the same as the actual value. Using the table above, this means that actual value was positive and the predicted value was also positive.

#### True Negative (TN): 
The predicted value also matches the actual value. In this case, it is for the negative class. The actual value is negative and the predicted value is negative.

#### False Positive (FP): 
Also called a Type I error, this is a misclassification such that the model predicted a positive class while the actual class is negative. Telling a man that he is pregnant is definitely a false positive.

#### False Negative (FN): 
Also another misclassification where the predicted value is negative and the actual value is positive. Another example will be telling a pregnant woman that she is not pregnant. FN is known as a Type II error.

In [42]:
new_prediction = log_reg.predict(normalised_test_df)
new_prediction[:5]

array(['3A', '2A', '2A', '3A', '3A'], dtype=object)

In [43]:
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_prediction, labels=['2A', '3A'])
cnf_mat

array([[33, 42],
       [55, 47]], dtype=int64)

### Accuracy

This is the ratio of the number of correctly predicted instances to the total number of instances. It is a commonly used metric suitable when the target classes are not imbalanced. A high accuracy does not necessarily mean that the model has high predicting power. Hence, depending on the task, it is important to not use only the accuracy metric because it does not provide enough information about the model.

$$
Accuracy = \frac{TP + TN}{TP + TN + FP + FN}
$$

In [44]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_prediction)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.45


### Precision

The ratio of correctly predicted instances of a class to the total number of items predicted by the model to be in that class is referred to as precision (known as Positive Predicted Value - PPV). This translates to the total percentage of the results obtained that are relevant. For the positive class, it is the ratio of true positives to the sum of true positives and false positives

$$
Precision = \frac{TP}{TP + FP}
$$

In [46]:
precision = precision_score(y_true=y_test, y_pred=new_prediction, pos_label='2A')
print(f'precision: {precision:.2f}')

precision: 0.38


### Recall

Known as the sensitivity of the model, recall gives a percentage of total relevant results correctly predicted by the model. It is the ratio of the true positives to the actual number of positives (true positives and false negatives).

$$
Recall = \frac{TP}{TP + FN}
$$

there is also a trade-off between precision and recall. It is impossible to maximise both metrics simultaneously because an increase in recall decreases precision. Identify which metric is important based on your task and optimise.

In [47]:
recall = recall_score(y_true=y_test, y_pred=new_prediction, pos_label='2A')
print(f"Recall: {recall:.2f}")

Recall: 0.44


### F1-Score

This metric is the harmonic mean of precision and recall that aims to have an optimal balance of both. The F1-Score is quite easy to use and can be focused on to maximize as opposed to maximizing precision and recall.

$$
F_1 = 2 * \frac{precision * recall}{precision + recall}
$$

In [48]:
f1 = f1_score(y_true=y_test, y_pred=new_prediction, pos_label='2A')
print(f"F1: {f1:.2f}")

F1: 0.40


### ROC Curve

The Receiver Operating Characteristics (ROC) curve is a probability curve that measures the performance of a classification model at different set thresholds. Recall also known as the True Positive Rate (TPR) is plotted on the y-axis against the False Positive Rate (FPR) on the x-axis.

The code examples above are not the optimal results that can be obtained with the model. Hyperparameter tuning can be performed to improve the model.

# Multiclass Classification